In [1]:
import importtestsmodule
import json
import kafka_client
import utils
import test_da

## Configuration and topics

### Config

`kafka_server` - set to `localhost:9094` (use `kafka:9092` if you run it inside a container)

### Topics

`sentiment` - sentiment analyser

### Credentials

Stored in a file `cred.json`. Some data acquirers require credentials.


In [ ]:
config=utils.load_config("../config.json")
cred_path = "../cred.twitter"

# Tests

## Test data acquirer - data acquisition

**Prerequisities**:
- The data acquirer listents at a topic `config['topics']['dataacquirer_update_topic']`
- Credentials in case of a twitter data acquirer or reddit data acquirer.

**Steps**

- Creates new job
- Waits for one message
- Checks that the message is in the a correct post
  - Success if true, fails otherwise
- Stops the running job

In [2]:
test_da.start_test("../config.json","../cred.json")

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3267, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-e15332cc9f29>", line 1, in <module>
    test_da.start_test("../config.json","../cred.json")
NameError: name 'test_da' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2018, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Jry\Anaconda3\lib\site-packages\IPython\core\ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "C:\Users\Jry\Anaconda3\lib\s

NameError: name 'test_da' is not defined

## Test job service registration

**Prerequisites**:

- TBA

**Steps**:

- TBA

## Test job service - job submit

**Prerequisites**:

- TBA

**Steps**:

- TBA

In [10]:
import sys
import json
import kafka_client
import uuid
import utils
import requests
from threading import Thread
import time
import traceback

class JmsTestCases:    
    def __init__(self, kafka, jms_registration_topic):
        self.jms_registration_topic = jms_registration_topic
        self.kafka = kafka

    def _register_acquirer(self, acquirer_component_id, acquirer_update_channel):
        registration_request_da ={
            "componentId" : acquirer_component_id,
            "componentType" : "DATA_ACQUIRER",
            "updateChannelName" : acquirer_update_channel,
            "inputChannelName" : "n/a" 
        }
        self.kafka.produce_request(registration_request_da, self.jms_registration_topic)
        return registration_request_da
    
    def _register_analyser(self, analyser_component_id, analyser_update_channel):
        registration_request_an ={
            "componentId" : analyser_component_id,
            "componentType" : "DATA_ANALYSER",
            "attributes" : {
                "outputFormat" :{
                    "o1" : "numberValue"
                }
            },
            "updateChannelName" : analyser_update_channel,
            "inputChannelName" : "test_input_channel_analyser" 
        }
        self.kafka.produce_request(registration_request_an, self.jms_registration_topic)
        return registration_request_an
        
    def _assert_data_acquirer_job_submit(self,
                                         topic_query,
                                         analyser_input_channel_name,
                                         acquirer_update_channel):
        print("Asserting data acquirer job config")
        job_config = self.kafka.get_message_from_topic(acquirer_update_channel)
        if job_config is None:
            raise Exception("No job config received")
        if "jobId" not in job_config:
            raise Exception("Job config has no job id")

        self._assert_value(job_config['attributes']['TopicQuery'], topic_query)
        self._assert_in(analyser_input_channel_name, job_config['outputChannelNames'])

        return job_config["jobId"]
        
    def _assert_analyser_job_submit(self,
                                   analyser_update_channel,
                                   job_id):
        print("Asserting analyser job config")
            
        job_config = self.kafka.get_message_from_topic(analyser_update_channel)
        if job_config is None:
            raise Exception("No job config received")
        self._assert_value(job_config['jobId'], job_id)
        
    def test(
        self,
        jms_submit_job_uri):
        
        current_uuid = str(uuid.uuid4())
        current_uuid = "f7713510-8122-4b05-835d-9a0a84609e76"
        
        print( "Registering new components")
        acquirer_component_id ="test_component_acquirer_"+current_uuid
        acquirer_update_channel = "topic_"+acquirer_component_id
        registration_request_da = self._register_acquirer(acquirer_component_id, acquirer_update_channel)
        
        analyser_component_id = "test_component_analyser_"+current_uuid
        analyser_update_channel = "topic_"+analyser_component_id        
        registration_request_an = self._register_analyser(analyser_component_id, analyser_update_channel)
        
        time.sleep(5)
        
        print("Submit job config")
        job_submit_request = {  
            "selectedDataAnalysers": [ analyser_component_id ],  
            "selectedDataAcquirers": [ acquirer_component_id ],  
            "topicQuery": "test query",  
            "jobName": "my test job",  
            "language": None,  
            "attributes": { "att1":"value1", "att2":"value2" }
        }
        data = json.dumps(job_submit_request)
        r = requests.post(jms_submit_job_uri, data)
        
        job_id = None
                
        print("Assert received job configs for each component")
        
        try:
            job_id = self._assert_data_acquirer_job_submit(
                job_submit_request["topicQuery"],
                registration_request_an['inputChannelName'],
                acquirer_update_channel
            )
        except Exception:
            return (False, traceback.format_exc())
             
        try:
            self._assert_analyser_job_submit(analyser_update_channel, job_id)
        except Exception as e:
            return (False, "Testing data analyser failed: {}".format(e))
        
        # TODO replay
                                    
    def _assert_value(self, a,b):
        if(a!=b):
            raise Exception("AssertError - Invalid value. actual:{} expected:{}".format(a,b))
            
    def _assert_in(self, a,b):
        if(a not in b):
            raise Exception("AssertError - Value not in array value:{} array:{}".format(a,b))


def start_test(config_path = "../config.json"):
    config=utils.load_config(config_path)
    kafka = kafka_client.KafkaClient(config['kafka']['kafka_server'])
    jms_registration = config['topics']['registration_channel_name']
    
    
    jms_tests = JmsTestCases(kafka,jms_registration)
    jms_submit_uri = config['uris']['jms_submit_uri']
    success,message = jms_tests.test(jms_submit_uri)
    if not success:
        print("FAIL", message)
    else :
        print("SUCCESS")
    

# if __name__ == "__main__":
#   if(len(sys.argv)==3):
#       config_path = sys.argv[1]
#       start_test(config_path);
#   else:
#       start_test();

start_test()




config loaded successfully
Registering new components
Submit job config
Assert received job configs for each component
Asserting data acquirer job config
Asserting analyser job config


TypeError: cannot unpack non-iterable NoneType object

In [8]:
config=utils.load_config("../config.json")
kafka = kafka_client.KafkaClient(config['kafka']['kafka_server'])
run_id = "f7713510-8122-4b05-835d-9a0a84609e76"
topic = "topic_test_component_analyser_{}".format(run_id)
topic = "topic_test_component_acquirer_{}".format(run_id)

while True:
    job_config = kafka.get_message_from_topic(topic)
    if job_config is None:
        print("nope")
        break
    print(job_config)
    

config loaded successfully
nope
